In [1]:
# 데이터 가공, 조작, 분석 라이브러리
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series, DataFrame
import pandas as pd

# 시각화 라이브러리
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

# 머신러닝 라이브러리
import sklearn

# 소수점 세 번째 자리까지 표시
%precision 3

'%.3f'

# 8.2 다중회귀

## 8. 2. 1. 자동차 가격 데이터 읽어 들이기

In [2]:
# 임포트
import requests, zipfile
import io

# 자동차 가격 데이터 가져오기
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data'
res = requests.get(url).content

# 데이터를 DataFrame 객체로 읽어 들이기
auto = pd.read_csv(io.StringIO(res.decode('utf-8')), header = None)

# 데이터 컬럼에 레이블 설정
auto.columns = ['symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration', 'num-of-doors',
                'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 'length', 'width', 'height',
                'curb-weight', 'engine-type', 'num-of-cylinders', 'engine-size', 'fuel-system', 'bore',
                'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']

In [3]:
print('자동차 데이터 형태:{}'.format(auto.shape))

자동차 데이터 형태:(205, 26)


In [4]:
auto.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


## 8. 2. 2. 데이터 정리

In [5]:
# 불필요한 데이터 제거

# 각 컬럼 데이터에 포함된 ? 개수 확인
auto = auto[['price', 'horsepower', 'width', 'height']]
auto.isin((['?'])).sum()

price         4
horsepower    2
width         0
height        0
dtype: int64

In [6]:
# ?를 NaN으로 전환하고 NaN이 있는 행 데이터 제거
auto = auto.replace('?', np.nan).dropna()
print('자동차 데이터 형태: {}'.format(auto.shape))

자동차 데이터 형태: (199, 4)


In [7]:
# 형 변환

print('데이터 형 확인(형 변환 전)\n{}\n'.format(auto.dtypes))

데이터 형 확인(형 변환 전)
price          object
horsepower     object
width         float64
height        float64
dtype: object



In [8]:
auto = auto.assign(price = pd.to_numeric(auto.price))
auto = auto.assign(horsepower = pd.to_numeric(auto.horsepower))
print('데이터 형 확인(형 변환 후)\n{}'.format(auto.dtypes))

데이터 형 확인(형 변환 후)
price           int64
horsepower      int64
width         float64
height        float64
dtype: object


In [9]:
# 상관관계 확인

auto.corr()

,price,horsepower,width,height
price,1.000000,0.810533,0.753871,0.134990
horsepower,0.810533,1.000000,0.615315,-0.087407
width,0.753871,0.615315,1.000000,0.309223
height,0.134990,-0.087407,0.309223,1.000000


## 8. 2. 3. 모델 구축과 평가

In [12]:
# 데이터 분할(훈련 데이터와 테스트 데이터)을 위한 임포트
from sklearn.model_selection import train_test_split

# 다중회귀 모델 구축을 위한 임포트
from sklearn.linear_model import LinearRegression

# 목표변수로 price를 지정, 설명변수로는 price 외의 다른 변수를 지정
X = auto.drop('price', axis = 1)
y = auto['price']

# 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state = 0)

# 다중회귀 클래스 초기화와 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 결정계수 출력
print('결정계수(train): {:.3f}'.format(model.score(X_train, y_train)))
print('결정계수(test):{:.3f}'.format(model.score(X_test, y_test)))

# 회귀계수와 절편 출력
print('\n회귀계수\n{}'.format(pd.Series(model.coef_, index = X.columns)))
print('절편: {:.3f}'.format(model.intercept_))

결정계수(train): 0.733
결정계수(test):0.737

회귀계수
horsepower      81.651078
width         1829.174506
height         229.510077
dtype: float64
절편: -128409.046


## 8. 2. 4. 모델 구축 및 모델 평가 과정 정리

In [ ]:
# Let's try
# 변수증가법, 변수감소법, 스텝와이즈법에 대해 조사해 보세요

In [ ]:
# 연습문제 8-1


In [ ]:
# Let's try
# 연습문제 8-1 데이터에서 목표변수를 price로 두고 

# 8. 3. 로지스틱 회귀

## 8. 3. 1. 로지스틱회귀 예

In [16]:
# 데이터 수집
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
res = requests.get(url).content

# 수집된 데이터를 DataFrame 객체로 읽어 들이기
adult = pd.read_csv(io.StringIO(res.decode('utf-8')), header = None)

# 데이터 컬럼에 레이블 설정
adult.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'martial-status',
                 'occupation', 'relationship', 'race', 'sex', 'capital-gain',
                 'capital-loss', 'hours-per-week', 'native-country', 'flg-50K']

# 데이터 형태와 결측값 개수 출력
print('데이터 형태: {}'.format(adult.shape))
print('결측값 수: {}'.format(adult.isnull().sum().sum()))

# 데이터의 처음 다섯 행 출력
adult.head()

데이터 형태: (32561, 15)
결측값 수: 0


,age,workclass,fnlwgt,education,education-num,martial-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,flg-50K
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## 8. 3. 2. 데이터 정리

In [17]:
adult.groupby('flg-50K').size()

flg-50K
 <=50K    24720
 >50K      7841
dtype: int64

In [19]:
# flg-50K 칼럼 값이 >50K이면 1, 아니면 0인 fin_flg 변수 추가 생성
adult['fin_flg'] = adult['flg-50K'].map(lambda x: 1 if x ==' >50K' else 0)
adult.groupby('fin_flg').size()

fin_flg
0    24720
1     7841
dtype: int64

## 8. 3. 3. 모델 구축과 평가

In [ ]:
from 